<a href="https://colab.research.google.com/github/Adrianus-Bagas/PerangkumanTeks/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perangkuman Teks Secara Ekstraktif Menggunakan BERT, GPT, dan TFIDF

## 1. Import beberapa libraries

In [ ]:
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 9.4 MB/s 
     |████████████████████████████████| 6.6 MB 40.4 MB/s 
     |████████████████████████████████| 120 kB 45.8 MB/s 


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from summarizer import Summarizer,TransformerSummarizer
from rouge import Rouge
import pandas as pd
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
teks = '''Cristiano Ronaldo merupakan salah satu pesepak bola terbaik di abad 21. Sepanjang kariernya, 
Cristiano Ronaldo mampu meraih prestasi-prestasi tertinggi di dunia sepak bola. 
Selama lebih dari dua dekade, Ronaldo telah membela empat tim yang berbeda, yaitu Sporting CP, Manchester United, Real Madrid, dan Juventus. 
Dari keempat tim itu, pemain berusia 37 tahun itu mampu memberikan gelar ke semua tim yang ia bela. 
Total sudah ada 34 trofi yang didapat oleh Ronaldo bersama empat tim tersebut. 
Selain prestasi untuk tim, pemain asal Portugal itu mampu meraih penghargaan di level individual. 
Total, Ronaldo sudah meraih lima kali dinobatkan sebagai pemain terbaik dunia lewat gelar Ballon d’Or. 
Jumlah penghargaan itu masih bisa bertambah mengingat Ronaldo saat ini masih aktif bermain.'''

In [ ]:
bert_model = Summarizer()
bert_summary = ''.join(bert_model(teks, min_length=60))
print(bert_summary)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Cristiano Ronaldo merupakan salah satu pesepak bola terbaik di abad 21. Sepanjang kariernya, 
Cristiano Ronaldo mampu meraih prestasi-prestasi tertinggi di dunia sepak bola.


In [ ]:
rouge = Rouge()
rouge_scores_bert = rouge.get_scores(teks, bert_summary, avg=True)

In [ ]:
rouge_scores_bert

{'rouge-1': {'r': 1.0, 'p': 0.2159090909090909, 'f': 0.3551401839951088},
 'rouge-2': {'r': 1.0, 'p': 0.1891891891891892, 'f': 0.3181818155061984},
 'rouge-l': {'r': 1.0, 'p': 0.2159090909090909, 'f': 0.3551401839951088}}

In [ ]:
pd.DataFrame(rouge_scores_bert)

,rouge-1,rouge-2,rouge-l
r,1.000000,1.000000,1.000000
p,0.215909,0.189189,0.215909
f,0.355140,0.318182,0.355140


In [ ]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(teks, min_length=60))
print(full)

Cristiano Ronaldo merupakan salah satu pesepak bola terbaik di abad 21. Total, Ronaldo sudah meraih lima kali dinobatkan sebagai pemain terbaik dunia lewat gelar Ballon d’Or.


In [ ]:
rouge_scores_gpt = rouge.get_scores(teks, full, avg=True)

In [ ]:
rouge_scores_gpt

{'rouge-1': {'r': 1.0, 'p': 0.2727272727272727, 'f': 0.42857142520408165},
 'rouge-2': {'r': 0.96, 'p': 0.21621621621621623, 'f': 0.35294117346993953},
 'rouge-l': {'r': 1.0, 'p': 0.2727272727272727, 'f': 0.42857142520408165}}

In [ ]:
pd.DataFrame(rouge_scores_gpt)

,rouge-1,rouge-2,rouge-l
r,1.000000,0.960000,1.000000
p,0.272727,0.216216,0.272727
f,0.428571,0.352941,0.428571


In [ ]:
def _create_frequency_table(text_string) -> dict:
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable

In [ ]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("indonesian"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [ ]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [ ]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [ ]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [ ]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [ ]:
def _score_sentences(tf_idf_matrix) -> dict:
    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [ ]:
def _find_average_score(sentenceValue) -> int:
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    average = (sumValues / len(sentenceValue))

    return average

In [ ]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [ ]:
def run_summarization(text):

    # 1 Sentence Tokenize
    sentences = sent_tokenize(text)
    total_documents = len(sentences)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(sentences)
    #print(freq_matrix)

    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.3 * threshold)
    return summary

In [ ]:
result = run_summarization(teks)
print(result)

 Jumlah penghargaan itu masih bisa bertambah mengingat Ronaldo saat ini masih aktif bermain.


In [ ]:
rouge_scores_tfidf = rouge.get_scores(teks, result, avg=True)

In [ ]:
rouge_scores_tfidf

{'rouge-1': {'r': 1.0, 'p': 0.13636363636363635, 'f': 0.23999999788800003},
 'rouge-2': {'r': 1.0, 'p': 0.10810810810810811, 'f': 0.1951219494586556},
 'rouge-l': {'r': 1.0, 'p': 0.13636363636363635, 'f': 0.23999999788800003}}

In [ ]:
pd.DataFrame(rouge_scores_tfidf)

,rouge-1,rouge-2,rouge-l
r,1.000000,1.000000,1.000000
p,0.136364,0.108108,0.136364
f,0.240000,0.195122,0.240000
